In [1]:
import numpy as np
from isaacgym import gymapi, gymtorch
from isaacgym.torch_utils import *
from tqdm import tqdm

Importing module 'gym_38' (/data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /data2/users/nk523/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.1.1
Device count 4
/data2/users/nk523/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /data2/users/nk523/.cache/torch_extensions/py38_cu121 as PyTorch extensions root...
Emitting ninja build file /data2/users/nk523/.cache/torch_extensions/py38_cu121/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module gymtorch...


ninja: no work to do.


In [2]:
from franka_panda_env import FrankaPandaEnv
from misc import wrap_angle
from motion_primitive_controller import MotionPrimitiveController
from position_controller import PositionController

In [3]:
env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False, headless=False)

Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: disabled
Loading asset 'urdf/franka_description/robots/franka_panda_gripper.urdf' from 'assets'
Creating 36 environments


In [4]:
motion_primitive_controller = MotionPrimitiveController(env.states["eef_pos"])
motion_primitive_controller.update(
    current_positions=env.states["eef_pos"],
    current_velocities=env.states["eef_vel"],
    cube_pos_relative=env.states["cube_pos_relative"],
    force=True,
)

[]

In [5]:
position_controller = PositionController(env)

In [ ]:
for _ in tqdm(range(20)):
    # Reset all environments
    env.reset_pose("default")
    env.reset_cube()
    motion_primitive_controller.dones *= False
    motion_primitive_controller.update(
        current_positions=env.states["eef_pos"],
        current_velocities=env.states["eef_vel"],
        cube_pos_relative=env.states["cube_pos_relative"],
        force=True,
        epsilon=0.5,
    )
    for _ in range(500):
        indices_done = motion_primitive_controller.update(
            current_positions=env.states["eef_pos"],
            current_velocities=env.states["eef_vel"],
            cube_pos_relative=env.states["cube_pos_relative"],
            epsilon=0.5,
        )
        position_controller.set_target(motion_primitive_controller.current_targets[:, :2])
        position_controller.step()
        env.reset_pose("default", indices_done)
        env.reset_cube(indices_done)
        motion_primitive_controller.dones *= False

  0%|                                                                                  | 0/20 [00:00<?, ?it/s]

In [ ]:
env.

In [ ]:
trajs = [item for sublist in motion_primitive_controller.trajectories for item in sublist]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np

In [ ]:
data = torch.stack([transition[0] for transition in trajs]).cpu().detach().numpy()
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]

In [ ]:
# Adjusting the color mapping based on the new requirements

# Center value for color mapping
z_center = 0.025

# Normalize Z values around the center (0.025) 
# Using the maximum absolute deviation from the center to ensure symmetry in the color scale
max_deviation = max(np.abs(z - z_center))
z_normalized_centered = (z - z_center) / max_deviation
z_normalized_centered = np.clip(z_normalized_centered, -1, 1)  # Ensuring values are within [-1, 1]

# Create a colormap that transitions from blue (negative) to gray (zero) to red (positive)
cmap_centered = plt.cm.get_cmap('coolwarm')

# Create the scatter plot with the new color mapping
plt.figure(figsize=(6, 6))
sc = plt.scatter(x, y, c=z_normalized_centered, cmap=cmap_centered, vmin=-1, vmax=1)

# Setting aspect ratio to 1:1
plt.gca().set_aspect('equal', adjustable='box')

# Adding a color bar
plt.colorbar(sc, label='Z value (Centered and Normalized)')

# Setting labels
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.title('Colored XY Plot Centered at Z=0.025')

circle = Circle((-0.5, 0), 0.75, color='black', fill=False)
plt.gca().add_patch(circle)

plt.grid()

# Show plot
plt.show()
